In [1]:
# necessary libraries

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()

import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
# to load dataset

df_raw = pd.read_csv('data\kc_housing_data.csv')
df = df_raw.copy()
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [4]:
# correlation coefficient between price and the other variables

df.corr(numeric_only=True)['price'].sort_values(ascending=False)

price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.256794
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
condition        0.036362
long             0.021626
id              -0.016762
zipcode         -0.053203
Name: price, dtype: float64

In [6]:
# to fit a multiple linear regression model

y = df['price'] # dependent variable
X = df[['bedrooms', 'bathrooms', 'view', 'grade']] # independent variables
X = sm.add_constant(X) # to add constant for the intercept
model_mr = sm.OLS(endog=y, exog=X)
results_mr = model_mr.fit()
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     5708.
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        15:36:35   Log-Likelihood:            -2.9981e+05
No. Observations:               21613   AIC:                         5.996e+05
Df Residuals:                   21608   BIC:                         5.997e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8.937e+05    1.3e+04    -68.898      0.000   -9.19e+05   -8.68e+05
bedrooms    1.855e+04   2186.988      8.484      0.000    1.43e+04    2.28e+04
bathrooms   5.401e+04   3304.360     16.346      0.000    4.75e+04    6.05e+04
view        1.164e+05   2349.627     49.556      0.000    1.12e+05    1.21e+05
grade       1.606e+05   2014.564     79.722      0.000    1.57e+05    1.65e+05
==============================================================================
Omnibus:                    18992.884   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1893016.549
Skew:                           3.778   Prob(JB):                         0.00
Kurtosis:                      48.222   Cond. No.                         66.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

>Since Adjusted $R^2$ = 0.514, the model fits data quite well. Furthermore, with significance level $\alpha$ = 0.05, the coefficients are all statistically significant because their p-value < $\alpha$. Then, the condition number is small enough.
Nevertheless, the model could be improved by adding another variable among the predictors.

>Because of a possible strong positive correlation between 'square footage' variables of the dataset and 'number of rooms' variables (i.e. 'bedrooms', 'bathrooms'), perhaps it's better to define a new variable about renovation of the houses to avoid multicollinearity.

In [7]:
# to create a new binary variable about renovation of the houses
# 1 if the house has been renovated, otherwise 0

df['yr_renovated'].unique()

array([   0, 1991, 2002, 2010, 1999, 1992, 2013, 1994, 1978, 2005, 2008,
       2003, 1984, 1954, 2014, 2011, 1974, 1983, 1945, 1990, 1988, 1957,
       1977, 1981, 1995, 2000, 1998, 1970, 1989, 2004, 1986, 2009, 2007,
       1987, 1973, 2006, 1985, 2001, 1980, 1971, 1979, 1997, 1950, 1969,
       1948, 2015, 1968, 2012, 1963, 1951, 1993, 1962, 1996, 1972, 1953,
       1955, 1982, 1956, 1940, 1976, 1946, 1975, 1958, 1964, 1959, 1960,
       1967, 1965, 1934, 1944], dtype=int64)

In [8]:
df['is_renovated'] = [1 if el != 0 else 0 for el in df['yr_renovated']]
df.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,is_renovated
11752,3790700070,20141121T000000,302500.0,4,2.50,1990,5511,2.0,0,0,...,1990,0,1994,0,98030,47.3585,-122.191,1850,6031,0
2023,3892500070,20140728T000000,1480000.0,3,3.50,4070,26000,2.0,0,0,...,4070,0,1991,0,98033,47.6590,-122.174,3770,26000,0
18408,5244801255,20150428T000000,705000.0,3,2.75,2260,4000,2.0,0,0,...,1540,720,1956,0,98109,47.6435,-122.353,2120,4000,0
12799,3905120540,20140618T000000,570000.0,4,2.50,2290,6738,2.0,0,0,...,2290,0,1996,0,98029,47.5714,-122.005,2100,6261,0
17663,5589900610,20140918T000000,559950.0,5,3.00,2730,9519,1.0,0,0,...,1670,1060,2014,0,98155,47.7504,-122.307,1150,9519,0


In [9]:
# to add the new variable among the predictors

y = df['price']
X = df[['bedrooms', 'bathrooms', 'view', 'grade', 'is_renovated']]
X = sm.add_constant(X)
model_mr = sm.OLS(endog=y, exog=X)
results_mr = model_mr.fit()
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     4709.
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        15:41:11   Log-Likelihood:            -2.9964e+05
No. Observations:               21613   AIC:                         5.993e+05
Df Residuals:                   21607   BIC:                         5.993e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -9.061e+05   1.29e+04    -70.318      0.000   -9.31e+05   -8.81e+05
bedrooms      1.878e+04   2169.733      8.655      0.000    1.45e+04     2.3e+04
bathrooms     5.095e+04   3282.351     15.524      0.000    4.45e+04    5.74e+04
view           1.12e+05   2343.360     47.783      0.000    1.07e+05    1.17e+05
grade         1.622e+05   2000.519     81.089      0.000    1.58e+05    1.66e+05
is_renovated   1.61e+05   8644.471     18.621      0.000    1.44e+05    1.78e+05
==============================================================================
Omnibus:                    18917.249   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1839698.818
Skew:                           3.761   Prob(JB):                         0.00
Kurtosis:                      47.568   Cond. No.                         66.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

>Adjusted $R^2$ has increased of about 1%. It's not so much, but condition number is still low and the coefficients are still statistically significant.

In [10]:
# to predict a price given the values of the predictors
# predictors = [constant, bedrooms, bathrooms, view, grade, is_renovated]

pred_price = results_mr.predict([1, 4, 2, 0, 8, 1])
print('Predicted price:', round(pred_price[0], 2), 'USD')

Predicted price: 729642.01 USD


In [11]:
# to verify the previous intuition about 'square footage' variables by adding 'sqft_living' as predictor

y = df['price']
X = df[['bedrooms', 'bathrooms', 'view', 'grade', 'is_renovated', 'sqft_living']]
X = sm.add_constant(X)
model_mr = sm.OLS(endog=y, exog=X)
results_mr = model_mr.fit()
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.584
Method:                 Least Squares   F-statistic:                     5057.
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        15:41:46   Log-Likelihood:            -2.9812e+05
No. Observations:               21613   AIC:                         5.963e+05
Df Residuals:                   21606   BIC:                         5.963e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -4.682e+05   1.43e+04    -32.838      0.000   -4.96e+05    -4.4e+05
bedrooms     -2.998e+04   2196.042    -13.654      0.000   -3.43e+04   -2.57e+04
bathrooms     -2.33e+04   3325.472     -7.006      0.000   -2.98e+04   -1.68e+04
view           8.99e+04   2218.677     40.522      0.000    8.56e+04    9.43e+04
grade         9.397e+04   2215.883     42.407      0.000    8.96e+04    9.83e+04
is_renovated   1.44e+05   8064.644     17.851      0.000    1.28e+05     1.6e+05
sqft_living    198.0627      3.472     57.041      0.000     191.257     204.869
==============================================================================
Omnibus:                    16145.340   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           952794.689
Skew:                           3.048   Prob(JB):                         0.00
Kurtosis:                      34.951   Cond. No.                     2.04e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

>As expected, condition number has increased enormously.

In [13]:
# correlation coefficient between sqft_living and some of the other predictors is very high

df[['bedrooms', 'bathrooms', 'grade', 'sqft_living']].corr()['sqft_living'].sort_values(ascending=False)

sqft_living    1.000000
grade          0.762704
bathrooms      0.754665
bedrooms       0.576671
Name: sqft_living, dtype: float64

In [14]:
# to define a multiple linear regression model in which dataset is randomly divided in training and testing data

y = df['price'] # target
X = df[['bedrooms', 'bathrooms', 'view', 'grade', 'is_renovated']] # predictors
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
# to fit the model to the training data

model_mr = sm.OLS(endog=y_train, exog=X_train)
results_mr = model_mr.fit()
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     3199.
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        15:44:13   Log-Likelihood:            -2.0029e+05
No. Observations:               14480   AIC:                         4.006e+05
Df Residuals:                   14474   BIC:                         4.006e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.793e+05   1.53e+04    -57.632      0.000   -9.09e+05   -8.49e+05
bedrooms      1.852e+04   2547.634      7.269      0.000    1.35e+04    2.35e+04
bathrooms     5.305e+04   3905.973     13.583      0.000    4.54e+04    6.07e+04
view          1.088e+05   2796.095     38.909      0.000    1.03e+05    1.14e+05
grade         1.582e+05   2371.964     66.690      0.000    1.54e+05    1.63e+05
is_renovated  1.656e+05   1.03e+04     16.004      0.000    1.45e+05    1.86e+05
==============================================================================
Omnibus:                    12150.334   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1069332.790
Skew:                           3.529   Prob(JB):                         0.00
Kurtosis:                      44.504   Cond. No.                         66.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# to write a function for a performance metric: Mean Absolute Error function

def mae(y, pred):
    return round(np.mean(np.abs(y - pred)), 2)

In [17]:
# to compare Training MAE to the MAE relative to the y_train mean

pred_train = results_mr.predict(X_train)

print('MAE relative to the average of y_train:', mae(y_train, np.mean(y_train)))
print('Training MAE:', mae(y_train, pred_train))

if mae(y_train, pred_train) < mae(np.mean(y_train), pred_train):
    print('Training MAE is better than MAE relative to the average of y_train')
else:
    print('Training MAE is worse than MAE relative to the average of y_train')

MAE relative to the average of y_train: 230714.87
Training MAE: 162253.46
Training MAE is better than MAE relative to the average of y_train


In [18]:
# to compare Testing MAE to the MAE relative to the y_test mean

pred_test = results_mr.predict(X_test)

print('MAE relative to the average of y_test:', mae(y_test, np.mean(y_test)))
print('Testing MAE:', mae(y_test, pred_test))

MAE relative to the average of y_test: 240532.88
Testing MAE: 168075.5


>Both Training MAE and Testing MAE are better than the same performance metric relative to the average of the given target values. Moreover, they are not so far each other. So, the model has not overfitted the data.  
Nevertheless, both Training and Testing MAE are very large and the Training Adjusted $R^2$ is not so high, just 0.525.   
Perhaps, the model could be improved by adding new appropriate variables as predictors.

In [19]:
# other performance metrics

# Mean Absolute Percentage Error

def mape(y, pred):
    return round(np.mean((np.abs(y - pred)/y))*100, 2)

# Root Mean Squared Error

def rmse(y, pred):
    return round(np.sqrt(np.mean((y - pred)**2)), 2)

In [20]:
mape_train = mape(y_train, pred_train)
mape_test = mape(y_test, pred_test)

print('Training MAPE:', mape_train, '%')
print('Testing MAPE:', mape_test, '%')

Training MAPE: 34.7 %
Testing MAPE: 34.95 %


> Training and Testing MAPE are very large.

In [21]:
rmse_train = rmse(y_train, pred_train)
rmse_test = rmse(y_test, pred_test)

print('Training RMSE:', rmse_train)
print('Testing RMSE:', rmse_test)

Training RMSE: 246005.52
Testing RMSE: 269549.52


In [22]:
# R-squared of the test predictions

# sum of squared residuals
ssres_test = np.sum((y_test - pred_test)**2)

# sum of squared deviations from the mean
sstot_test = np.sum((y_test - np.mean(y_test))**2)

# R^2
rsqr_test = 1 - (ssres_test / sstot_test)

print('R-squared of test set:', rsqr_test)

R-squared of test set: 0.5147513549023037


In [23]:
# Adjusted R**2 for the test set
# Adjusted R**2 = 1 - (1 - R**2)(N - 1)/(N - p - 1)

n = len(y_test) # I could have written X_test.shape[0] equivalently
p = X_test.shape[1] # we have 6 predictors, not only 5, because we have added a constant to build the model

adj_rsqr_test = 1 - ((1 - rsqr_test)*(n - 1) / (n - p - 1))


print('Adjusted R-squared of test set:', adj_rsqr_test)

Adjusted R-squared of test set: 0.5143427818079189


In [24]:
print('Adjusted R-squared < R-squared:', adj_rsqr_test < rsqr_test)

Adjusted R-squared < R-squared: True


In [25]:
# to add new predictors to improve the model

df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'is_renovated'],
      dtype='object')

In [27]:
# to add floors, waterfront and condition

y = df['price']
X = df[['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'is_renovated']]
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
model_mr = sm.OLS(endog=y_train, exog=X_train)
results_mr = model_mr.fit()
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     2237.
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        15:59:07   Log-Likelihood:            -1.9985e+05
No. Observations:               14480   AIC:                         3.997e+05
Df Residuals:                   14471   BIC:                         3.998e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.115e+06   1.95e+04    -57.105      0.000   -1.15e+06   -1.08e+06
bedrooms      1.271e+04   2504.575      5.076      0.000    7803.411    1.76e+04
bathrooms     6.796e+04   4013.371     16.933      0.000    6.01e+04    7.58e+04
floors       -3.605e+04   4490.785     -8.028      0.000   -4.49e+04   -2.72e+04
waterfront    5.231e+05    2.6e+04     20.095      0.000    4.72e+05    5.74e+05
view           7.86e+04   2953.251     26.616      0.000    7.28e+04    8.44e+04
condition     5.952e+04   3196.360     18.621      0.000    5.33e+04    6.58e+04
grade         1.684e+05   2360.437     71.332      0.000    1.64e+05    1.73e+05
is_renovated  1.666e+05   1.01e+04     16.541      0.000    1.47e+05    1.86e+05
==============================================================================
Omnibus:                    12000.224   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1142654.763
Skew:                           3.428   Prob(JB):                         0.00
Kurtosis:                      45.976   Cond. No.                         125.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
pred_train = results_mr.predict(X_train)

print('Training MAE:', mae(y_train, pred_train))
print('Training MAPE:', mape(y_train, pred_train), '%')


Training MAE: 157981.36
Training MAPE: 34.41 %


In [30]:
pred_test = results_mr.predict(X_test)

print('Testing MAE:', mae(y_test, pred_test))
print('Testing MAPE:', mape(y_test, pred_test), '%')

Testing MAE: 163517.71
Testing MAPE: 34.67 %


>Adjusted $R^2$ has increased, while condition number is still low and all the p-value of the coefficients are less than the significance level $\alpha$ = 0.05.  
Training and Testing performance metrics are very similar, so that the model has not overfitted the training data.  
Nevertheless, a MAPE of about 34% is too high.